Find which cysteines are near binding sites using primary sequences and UniProt annotations

In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib
import math
from matplotlib.pyplot import figure
import seaborn as sns
import string
from matplotlib import pyplot as plt
import plotly.io as pio
from matplotlib_venn import venn3
from matplotlib_venn import venn2, venn2_circles
from matplotlib import cm
from matplotlib.colors import ListedColormap
from venn import venn
import plotly.express as px

In [ ]:
date = '220920'

In [ ]:
cd = os.getcwd()
cd

# Read UniProt Binding Site Annotations

In [ ]:
bs_df = pd.read_csv('220908_uniprot_bs_identifiers.csv')

# Read CysDB IDs

In [ ]:
c_df = pd.read_excel('Table_S1.xlsx', sheet_name = 'Fig1D')

In [ ]:
c_df['is_bs'] = np.where(c_df['cysteineid'].isin(c_bs_ids), 1, 0)

In [ ]:
non_bs_c_df = c_df[c_df['is_bs'] == 0]
non_bs_c_ids = list(non_bs_c_df['cysteineid'].unique())

In [ ]:
c_df['check_bs'] = np.where(c_df['cysteineid'].isin(non_bs_c_ids), 1, 0)

In [ ]:
cysdb_w_u_bs = c_df[c_df['check_bs'] == 1]

In [ ]:
cysdb_w_u_bs['cysteineid'] = cysdb_w_u_bs['cysteineid'].map(lambda x: str(x).replace(' ', ''))

In [ ]:
cysdb_w_u_bs['resid'] = cysdb_w_u_bs['cysteineid'].map(lambda x: str(x).split('_C')[-1].strip())

In [ ]:
# find which cysteines are within +/-10 amino acids from an annotated UniProt binding site

def get_range(df, u_df):
    bs_neighbor = []
    count = 0
    bs_neighbors = []
    
    # iterate through cysdb
    for index, row in df.iterrows():
        
        # get identifiers
        pro = row['proteinid']
        cys = row['cysteineid'].replace(' ', '')
        aa = int(row['resid'])
        
        subset_u_df = u_df[u_df['Entry'] == pro]
        bs_ids = list(subset_u_df['Site'])
        bs_identifiers = list(subset_u_df['bs_id'])
        bs_neighbors.append(bs_ids)
        
        current_bs_ids = []
        
        for i in range(len(bs_ids)):
            u_lim = int(bs_ids[i]) + 10
            l_lim = int(bs_ids[i]) - 10
            
            if (aa >= l_lim) & (aa <= u_lim):
                current_bs_ids.append(bs_identifiers[i])
                count += 1
                
        bs_neighbor.append(current_bs_ids)
        
    return bs_neighbor, bs_neighbors

In [ ]:
near_bs_neighbors, bs_neighbor_list = get_range(cysdb_w_u_bs, bs_df)

In [ ]:
cysdb_w_u_bs['near_bs_neighbors'] = near_bs_neighbors

In [ ]:
cysdb_w_u_bs['near_bs_neighbors_list'] = bs_neighbor_list

In [ ]:
cysdb_w_u_bs['near_bs_ps'] = cysdb_w_u_bs['near_bs_neighbors'].str.len()

In [ ]:
cysdb_w_u_bs['near_bs_1D'] = np.where(cysdb_w_u_bs['near_bs_ps'] >= 1, 1, 0) 

In [ ]:
cysdb_w_u_bs['near_bs_neighbors'] = [';'.join(map(str, l)) for l in cysdb_w_u_bs['near_bs_neighbors']]

In [ ]:
c_near_bs_df = cysdb_w_u_bs[cysdb_w_u_bs['near_bs_1D'] == 1]

In [ ]:
c_near_bs = list(c_near_bs_df['cysteineid'].unique())

In [ ]:
merged_c_df = pd.merge(c_df, c_near_bs_df[['cysteineid', 'near_bs_1D', 'near_bs_neighbors']], on = 'cysteineid', how = 'left')

In [ ]:
# yes if cysteine is an annotated binding site or proximal to a binding site

def get_bs_label(df):
    
    label = []
    
    df = df.fillna(0)
    
    for index, row in df.iterrows():
        is_bs = int(row['is_bs'])
        near_bs = int(row['near_bs_1D'])
        
        if ((is_bs + near_bs) >= 1):
            label.append('yes')
        else:
            label.append(None)
            
    return label

In [ ]:
bs_labels = get_bs_label(merged_c_df)

In [ ]:
merged_c_df['bs_category'] = bs_labels

In [ ]:
lig_df = merged_c_df[merged_c_df['ligandable'] == 'yes']

In [ ]:
rxt_df = merged_c_df[merged_c_df['hyperreactive'] == 'yes']

In [ ]:
merged_c_df = merged_c_df.drop(columns = ['check_bs'])

In [ ]:
merged_c_df.to_csv(date + '_cysdb_binding_sites.csv', index = False)